## Домашнее задание № 11. Pandas. Продолжение.

### Задание 1

- [x] Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:


1. оценка 2 и меньше - низкий рейтинг
2. оценка 4 и меньше - средний рейтинг
3. оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [46]:
df = pd.read_csv('DATA/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [31]:
def rating_claster(row):
    """Создает классификацию по среднему рейтингу"""
    if 0 <= row['rating'] <= 5:
        if row['rating'] >= 4.5:
            return "Высокий"
        elif row['rating'] <= 2:
            return "Низкий"
        else:
            return "Средний"
    else:
        return "rating error"

In [32]:
film_ratings = df[['movieId','rating']].groupby('movieId').agg({'rating':'mean'})

film_ratings['rating_group'] = film_ratings.apply(rating_claster, axis = 1)
film_ratings.sort_values(by=['rating'], ascending = False).head(10)

,rating,rating_group
movieId,,
163949,5.0,Высокий
71180,5.0,Высокий
51471,5.0,Высокий
52617,5.0,Высокий
53887,5.0,Высокий
54251,5.0,Высокий
54328,5.0,Высокий
55555,5.0,Высокий
56869,5.0,Высокий


### Задание 2

- [x] Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

In [43]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [44]:
df = pd.read_csv('DATA/keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [45]:
def geo_region(row):
    """Создает классификацию по региону, если в кодовом слове содержится город из geo_data"""
    key_ = [key for key, value in geo_data.items() if row['keyword'].lower() in value]
    if key_:
        return key_[0]
    else:
        return 'undefined'

    
# Тестируем работу функции. Создаем тестовый датафрейм и смотрим работу для разных случаев написания и разных keyword. 
df_test = pd.DataFrame({ 'keyword': ['вк','ВК','одноклассники','МОСКВА','Питербург','петербург','сахалин','ХаБаРоВсК'], 
                         'shows': [2,2,3,4,150,5,6,7] })
df_test['region'] = df_test.apply(geo_region, axis = 1)
df_test
    

,keyword,shows,region
0,вк,2,undefined
1,ВК,2,undefined
2,одноклассники,3,undefined
3,МОСКВА,4,Центр
4,Питербург,150,undefined
5,петербург,5,Северо-Запад
6,сахалин,6,Дальний Восток
7,ХаБаРоВсК,7,Дальний Восток


In [42]:
%%time

df['region'] = df.apply(geo_region, axis = 1)
df.head()

Wall time: 3.72 s


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


### Задание 3

- [ ] Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
 * для каждой строки пройдите по всем годам списка years
 * если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 * если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [47]:
df = pd.read_csv('DATA/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [48]:
df2 = pd.read_csv('DATA/movies.csv')
df2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [57]:
import numpy as np
import re

In [131]:
def release_year(row):
    """Вытаскивает год выпуска из года фильма и возвращает его для каждой строки. Если не нашел - ставит 1900"""
    pattern = r'\((\d{4}).*\)'
    text = row['title']
    result = re.search(pattern, text)
    if result:
        return int(result.group(1))
    else:
        return 1900

In [132]:
# тестируем функцию

df_test = pd.DataFrame({ 'movieId': [1,2,3],
                         'title': ['Toy Story (1996)','Jumanji (1997)','Grumpier Old Men (1993)'],
                         'genres': ['Adventure|Animation|Children|Comedy|Fantasy','Adventure|Children|Fantasy','Comedy|Romance']})


df_test['release_year'] = df_test.apply(release_year, axis = 1)
df_test 

,movieId,title,genres,release_year
0,1,Toy Story (1996),Adventure|Animation|Children|Comedy|Fantasy,1996
1,2,Jumanji (1997),Adventure|Children|Fantasy,1997
2,3,Grumpier Old Men (1993),Comedy|Romance,1993


In [135]:
# применяем функцию ко второй таблице

df2['release_year'] = df2.apply(release_year, axis = 1)
df2.sort_values(by='release_year').head()

,movieId,title,genres,release_year
9124,164979,"Women of '69, Unboxed",Documentary,1900
9118,162376,Stranger Things,Drama,1900
9063,151307,The Lovers and the Despot,(no genres listed),1900
9017,143410,Hyena Road,(no genres listed),1900
6119,32898,"Trip to the Moon, A (Voyage dans la lune, Le) ...",Action|Adventure|Fantasy|Sci-Fi,1902


In [144]:
# Усредняем оценки пользователей по фильмам. Соединяем таблицы с рейтингами и годами и группируем по году релиза
avg_rating = df[['movieId','rating']].groupby('movieId').agg({'rating':'mean'}).reset_index()

finalta = pd.merge(avg_rating, df2, on='movieId').groupby('release_year').agg({'rating':'mean','movieId':'count'}).sort_values(by='rating', ascending = False).reset_index()
finalta.head()

,release_year,rating,movieId
0,1921,4.387500,5
1,1924,4.360000,5
2,1902,4.333333,1
3,1928,4.292143,5
4,1917,4.250000,1
